In [1]:
# Import required libraries
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

/Users/mhh/CodeServer/RevAnalytix/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Define the model and tokenizer
MODEL = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)

In [3]:
# Load the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
# Define a function to preprocess text
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [15]:
# Define a function to perform sentiment analysis
def perform_sentiment_analysis(text):
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    
    # Rank the sentiment labels by their scores
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    sentiments = []
    
    # Print labels and scores
    for i in range(scores.shape[0]):
        label = config.id2label[ranking[i]]
        score = scores[ranking[i]]
        sentiments.append((label, np.round(float(score), 4)))
    
    return sentiments

In [16]:
# Get input from the user
user_input = input("Enter your review: ")

# Perform sentiment analysis
sentiments = perform_sentiment_analysis(user_input)

# Print the sentiment analysis results
print("\nSentiment Analysis Results:")
for i, (label, score) in enumerate(sentiments):
    print(f"{i+1}) {label}: {score}")

Enter your review:  The app is wonderful! I love the characters, and everything. But what I don't like is the characters forgetting things to easily. The AI's keep repeating, and it gets annoying!!! There are a lot of bugs, but overall the app is good. I also think they need to make a kids ver, and an adult ver! Also not to mention the amount of times the server goes down and I have to wait for hours!!! I don't have $10 to pay for y'all's Talkie+, it's outrageous!!! There should be better severs!



Sentiment Analysis Results:
1) negative: 0.5088
2) positive: 0.3315
3) neutral: 0.1597


In [17]:
# Define a function to perform sentiment analysis and capture concerns
def perform_sentiment_and_concern_analysis(text):
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    
    # Define positive keywords and concern keywords
    positive_keywords = ["happy", "great", "excellent", "awesome"]
    concern_keywords = ["issue", "problem", "concern", "trouble", "complaint"]
    
    # Check for the presence of keywords
    detected_sentiment = None
    detected_concerns = []
    
    for keyword in positive_keywords:
        if keyword in text:
            detected_sentiment = "Positive"
            break
    
    for keyword in concern_keywords:
        if keyword in text:
            detected_concerns.append(keyword)
    
    if detected_sentiment:
        return detected_sentiment, detected_concerns, scores
    else:
        # If no sentiment keywords found, rank the sentiment labels by their scores
        ranking = np.argsort(scores)
        ranking = ranking[::-1]
        sentiments = []
        
        # Print labels and scores
        for i in range(scores.shape[0]):
            label = config.id2label[ranking[i]]
            score = scores[ranking[i]]
            sentiments.append((label, np.round(float(score), 4)))
        
        return sentiments, detected_concerns, scores

In [18]:
# Get input from the user
user_input = input("Enter your review: ")

# Perform sentiment and concern analysis
result, detected_concerns, scores = perform_sentiment_and_concern_analysis(user_input)

# Print the sentiment analysis results and detected concerns
if isinstance(result, str):
    print("Sentiment:", result)
else:
    print("\nSentiment Analysis Results:")
    for i, (label, score) in enumerate(result):
        print(f"{i+1}) {label}: {score}")

if detected_concerns:
    print("\nDetected Concerns:", ", ".join(detected_concerns))

Enter your review:  The app is wonderful! I love the characters, and everything. But what I don't like is the characters forgetting things to easily. The AI's keep repeating, and it gets annoying!!! There are a lot of bugs, but overall the app is good. I also think they need to make a kids ver, and an adult ver! Also not to mention the amount of times the server goes down and I have to wait for hours!!! I don't have $10 to pay for y'all's Talkie+, it's outrageous!!! There should be better severs!



Sentiment Analysis Results:
1) negative: 0.5088
2) positive: 0.3315
3) neutral: 0.1597
